# 1. Point projection and coordinate transformation

### Import Librarys

In [1]:
import cv2
import numpy as np
import os

### Import images

In [2]:


# Dynamische Pfaderstellung basierend auf dem aktuellen Notebook-Verzeichnis
notebook_dir = os.getcwd()
image_left_path = os.path.join(notebook_dir, 'Additional_files', 'Stereo_images', 'ConstructionSiteLeft', 'image0110_c0.pgm')
image_right_path = os.path.join(notebook_dir, 'Additional_files', 'Stereo_images', 'ConstructionSiteRight', 'image0110_c1.pgm')

image_left = cv2.imread(image_left_path, cv2.IMREAD_GRAYSCALE)
image_right = cv2.imread(image_right_path, cv2.IMREAD_GRAYSCALE)

# Kontrast für Anzeige erhöhen
image_left_display = cv2.normalize(image_left, None, 0, 255, cv2.NORM_MINMAX)
image_right_display = cv2.normalize(image_right, None, 0, 255, cv2.NORM_MINMAX)

### Get Pixels by mouse click

In [3]:
USE_MOUSE = False
pixel_left = None
pixel_right = None

if USE_MOUSE:
    def mouse_callback(event, x, y, flags, param):
        global pixel_left, pixel_right
        if event == cv2.EVENT_LBUTTONDOWN:
            img, name = param
            print(f"{name}: Klick bei ({x}, {y}) ")
            if name == 'Left Image':
                pixel_left = [x, y]
            elif name == 'Right Image':
                pixel_right = [x, y]

    cv2.namedWindow('Left Image')
    cv2.setMouseCallback('Left Image', mouse_callback, param=(image_left, 'Left Image'))
    cv2.namedWindow('Right Image')
    cv2.setMouseCallback('Right Image', mouse_callback, param=(image_right, 'Right Image'))

    cv2.imshow('Left Image', image_left_display)
    cv2.imshow('Right Image', image_right_display)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

else:
    pixel_left = [250, 250]
    pixel_right = [238, 250]

print("pixel_left:", pixel_left)
print("pixel_right:", pixel_right)

pixel_left: [250, 250]
pixel_right: [238, 250]


## Solution

### Calculate disparity

In [4]:
disparity = pixel_left[0] - pixel_right[0]
print("Disparity:", disparity)

Disparity: 12


### Read Camera parameters

In [5]:
camera_txt_path = os.path.join(notebook_dir, 'Additional_files', 'Camera.txt')
with open(camera_txt_path, 'r') as f:
    camera_txt_content = f.read()

internal_params = {}
external_params = {}

section = None
for line in camera_txt_content.strip().split('\n'):
    line = line.strip()
    if line == '[INTERNAL]':
        section = 'internal'
    elif line == '[EXTERNAL]':
        section = 'external'
    elif '=' in line and section:
        key, value = line.split('=', 1)
        value_clean = value.split('#', 1)[0].strip()
        try:
            value_num = float(value_clean)
        except ValueError:
            value_num = value_clean  
        if section == 'internal':
            internal_params[key.strip()] = value_num
        elif section == 'external':
            external_params[key.strip()] = value_num

print("internal_params:", internal_params)
print("external_params:", external_params)

internal_params: {'F': 820.428, 'SX': 1.0, 'SY': 1.000283, 'X0': 305.278, 'Y0': 239.826}
external_params: {'B': 0.308084, 'LATPOS': -0.07, 'HEIGHT': 1.26, 'DISTANCE': 2.0, 'TILT': 0.06, 'YAW': 0.01, 'ROLL': 0.0}


### Coordinate Calculation Camera


In [6]:
z = (internal_params['F'] * external_params['B']) / disparity
x = ((pixel_left[0] - internal_params['X0']) * internal_params['SX'] * z) / internal_params['F']
y = ((pixel_left[1] - internal_params['Y0']) * internal_params['SY'] * z) / internal_params['F']

K = np.array([
    [internal_params['F'] / internal_params['SX'], 0, internal_params['X0']],
    [0, internal_params['F'] / internal_params['SY'], internal_params['Y0']],
    [0, 0, 1]
])
print("Kameramatrix K:\n", K)

p_c = np.array([x, y, z])
print("Point Camera:", p_c)

Kameramatrix K:
 [[820.428        0.         305.278     ]
 [  0.         820.19588456 239.826     ]
 [  0.           0.           1.        ]]
Point Camera: [-1.41918895  0.26127781 21.063395  ]


### Shift from Camera to World

In [7]:
p_c_din = np.array([p_c[2], -p_c[0], -p_c[1]]) # Beim letzten - bin ich mir nicht ganz sicher
print("Point World (DIN 70000):", p_c_din)


Point World (DIN 70000): [21.063395    1.41918895 -0.26127781]


### Rotation matrix calculation

In [8]:
roll = external_params['ROLL']    
pitch = external_params['TILT']   
yaw = external_params['YAW']      

Rx = np.array([
    [1, 0, 0],
    [0, np.cos(roll), -np.sin(roll)],
    [0, np.sin(roll), np.cos(roll)]
])

Ry = np.array([
    [np.cos(pitch), 0, np.sin(pitch)],
    [0, 1, 0],
    [-np.sin(pitch), 0, np.cos(pitch)]
])

Rz = np.array([
    [np.cos(yaw), -np.sin(yaw), 0],
    [np.sin(yaw), np.cos(yaw), 0],
    [0, 0, 1]
])

R_wc = Rz @ Ry @ Rx # Weiß nicht, ob man das nicht noch invertieren muss
R_cw = np.linalg.inv(R_wc)
print("Rotationsmatrix R:\n", R_cw)

Rotationsmatrix R:
 [[ 9.98150630e-01  9.98183903e-03 -5.99640065e-02]
 [-9.99983333e-03  9.99950000e-01  0.00000000e+00]
 [ 5.99610083e-02  5.99630071e-04  9.98200540e-01]]


### Translation vector

In [9]:

t_wc_din = np.array([       # Weiß nicht, ob man das nicht noch invertieren muss
    [external_params['DISTANCE']],   
    [external_params['LATPOS']],     # Weiß nicht, ob da noch die Base line dazu muss
    [external_params['HEIGHT']]      
])

t_cw_din = -t_wc_din

print("Verschiebungsvektor t:", t_cw_din)

Verschiebungsvektor t: [[-2.  ]
 [ 0.07]
 [-1.26]]


### Apply Rotation and Translation

In [10]:

p_w_din = R_cw @ p_c_din + t_cw_din.T
print("3D-Punkt in Weltkoordinaten:", p_w_din)

3D-Punkt in Weltkoordinaten: [[19.05427437  1.27848755 -0.25697426]]


## Backward calculation


In [11]:
p_w_din = np.append(p_w_din, 1)



In [12]:
Rt_wc = np.hstack((R_wc, t_wc_din))

p_w = Rt_wc @ p_w_din.T

p_c = np.array([-p_w[1], -p_w[2], p_w[0]])
print("3D-Punkt in Kamerakoordinaten:", p_c)


3D-Punkt in Kamerakoordinaten: [-1.39846623  0.13908247 20.99084288]


In [13]:
p_c = np.dot(K, p_c)

pixel_c = np.array([int(p_c[0] / p_c[2]), int(p_c[1] / p_c[2])])

print("Pixel in Bildkoordinaten:", pixel_c)
print("Ursprünglicher Punkt:", pixel_left)

Pixel in Bildkoordinaten: [250 245]
Ursprünglicher Punkt: [250, 250]
